In [1]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample
from cmath import nan

In [2]:

batch_name='batch 1' #batch name - make sure the folder is structured as followes: (the test results/data doesnt have to be there)

#parent name: 
# -pilot_24_hours: 
#   -batch name (folder)
#       -data (folder)
#       -Batch_encoding_batch_results.csv 
#       -Batch_testing_batch_results.csv 
#       -Batch_workers.csv


qualification_name_for_testin='UPDATE-eligible for memory testing 1 week'  #note that the -space- after the title is improtant as for some reason this is how the qualification name is defined
qualification_name_for_entire_experiment='UPDATE-completed memory rep'

#all data location is relational to the location of this jupiter notebook:
batch_data_location=path.Path.cwd().parent
batch_data_location = batch_data_location / 'data' / 'contrast'/'pilot_one_week'


if (batch_data_location / batch_name).is_dir():
    batch_data_location= batch_data_location / batch_name
    print('Batch found',batch_data_location)
else: 
    print(f'PROBLEM FOUND: the requested batch name {batch_name} does not apear in the folder: {batch_data_location}')


PATH_TO_DATA = batch_data_location / 'data'
if not PATH_TO_DATA.exists():
    print('There is no data folder in the requested location',batch_data_location / 'data')
else: 
    print('Data folder found',batch_data_location / 'data')




Batch found c:\Users\User\Desktop\visualpilot\Visual-memory-task\data\contrast\pilot_one_week\batch 1
Data folder found c:\Users\User\Desktop\visualpilot\Visual-memory-task\data\contrast\pilot_one_week\batch 1\data


In [3]:
#this section loads the shelf dict in the state it is after finishing the testing session for this batch. 
import json
shelf_dict_after_test_name=batch_data_location / 'shelf after encoding session closed.txt' #define the name of the relevant shelf for this stage
with open(shelf_dict_after_test_name) as f:
    data = f.read()
shelf_dict = json.loads(data)
shelf_dict

{'A7VA2Y4H6U31O': [0, 1670945335008],
 'A2L18Q7871EONK': [1, 1670945472029],
 'A38C544PT5L6C7': [2, 1670945508029, 1670945818039],
 'A2C4Y3H6AI7TOC': [999, 999, 999],
 'A3R0S4QK7VY0QA': [3, 1670945605406],
 'A8C3WNWRBWUXO': [4, 1670945758899],
 'A37INK0K3GRFGL': [999, 999, 999],
 'A27DJWJG2RN67O': [999, 999, 999],
 'A304UJAE051J89': [5, 1670945807462],
 'A2PBFKNWWIIAO8': [6, 1670945922618],
 'A2D85WX82D81GJ': [7, 1670945930962, 1670946488104],
 'A3R8SCC5Q0OE7W': [8, 1670946068317],
 'A2K454H8X0MQ6': [999, 999, 999],
 'A12TWJFR4B22X4': [9, 1670946235280],
 'A29UY77DJOCTRB': [999, 999, 999],
 'A3FGCKDXG0D6PB': [11,
  1670946371142,
  1670946819241,
  1670947949899,
  1670949700945],
 'A3BI98I9V2FL0T': [10, 1670946643322],
 'A1DT7XOW0ISK0K': [12, 1670946674225, 1670947134247, 1670947196277],
 'A2UOURWPFGV8A4': [999, 999, 999],
 'AMW28M2VV2677': [999, 999, 999],
 'AJF954955YWZG': [15, 1670946923359],
 'AV0KMH4HKQBGK': [14, 1670947073106],
 'A3ECPU0FSUCJCM': [999, 999, 999],
 'A1LLBT92I9VVC

In [5]:
#this section reads all the relevant files: 
if (batch_data_location / 'Batch_encoding_batch_results.csv').exists():
    encoding_df=pd.read_csv(batch_data_location / 'Batch_encoding_batch_results.csv')
else: 
    print(f'no "Batch_encoding_batch_results.csv" was found in data')

if (batch_data_location / 'Batch_testing_batch_results.csv').exists():
    testing_df=pd.read_csv(batch_data_location / 'Batch_testing_batch_results.csv')
else: 
    print(f'no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)')
if (batch_data_location / 'Batch_workers.csv').exists():
    workers_df=pd.read_csv(batch_data_location / 'Batch_workers.csv')
else:
    print('No AMAZON worker list was found: please make sure to upload one and name it correctly')



no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)


In [6]:
workers_df.columns

Index(['Worker ID', 'Link to Individual Worker Page',
       'Number of HITs approved or rejected - Lifetime',
       'Number of HITs approved - Lifetime', 'Your Lifetime approval rate',
       'Number of HITs approved or rejected - Last 30 days',
       'Number of HITs approved - Last 30 days',
       'Your Last 30 days approval rate',
       'Number of HITs approved or rejected - Last 7 days',
       'Number of HITs approved - Last 7 days',
       'Your Last 7 days approval rate', 'CURRENT-took part in a tapseq task',
       'UPDATE-took part in a tapseq task', 'CURRENT-participated in the past',
       'UPDATE-participated in the past',
       'CURRENT-eligible for next step memory rep',
       'UPDATE-eligible for next step memory rep',
       'CURRENT-eligible for memory testing 1 week',
       'UPDATE-eligible for memory testing 1 week',
       'CURRENT-completed memory rep', 'UPDATE-completed memory rep',
       'CURRENT-completed memory encoding 24h',
       'UPDATE-completed m

In [7]:
def find_largest_consequtive_repetition(responses_array):
#add a stop value so the loop will use the logic also if the end of the responses is all repeating: 
    responses_array=np.append(responses_array,[99])
    longest_rep_dict=dict()
    last_input=responses_array[0]
    consequtive_rep_counter=0
    for response in responses_array[1:]: 
        if response==last_input: #if consequtive rep: 
            consequtive_rep_counter=consequtive_rep_counter+1 #add to counter
        else: #once a new entry is in the vector - store the counter rep information 
            if last_input in longest_rep_dict.keys(): #if it was allready stored in our dictionary
                if longest_rep_dict[last_input]<consequtive_rep_counter: #replace the counter only if it is higher than what stored in dict
                    longest_rep_dict[last_input]=consequtive_rep_counter
            else: #this entry is not yet in the dictionary: add it 
                longest_rep_dict[last_input]=consequtive_rep_counter
            
            consequtive_rep_counter=0
            last_input=response
    return max(longest_rep_dict.values())

In [8]:
def process_worker_results(PATH_TO_BATCH,subject_name):
    cur_sub=pd.read_csv(PATH_TO_BATCH / subject_name)
    sub_demographics=cur_sub[['workID','Age','Gender']].iloc[0]


    #extract demo related information: 

    #if participant did not click on the arrow during the demo a 'demo_encoding_response.rt' colmumn wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('demo_encoding_response.rt' in cur_sub.columns):
        cur_sub['demo_encoding_response.rt']=np.nan
    demo_columns=['demo_encoding_loop.thisTrialN','DemoImage','DemoCorrect','demo_encoding_response.rt','demo_encoding_response.keys']

    sub_demo_information=cur_sub[demo_columns]
    empty_inds=sub_demo_information.loc[sub_demo_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_information=sub_demo_information.drop(index=empty_inds)


    #extract the demo test columns: 
    demo_test_columns=['demo_test_response.keys','demo_test_response.corr','demo_test_response.rt','demo_test_loop.thisTrialN','DemoImage1','DemoImage2','DemoCorrectTest']
    sub_demo_test_information=cur_sub[demo_test_columns]
    empty_inds=sub_demo_test_information.loc[sub_demo_test_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_test_information=sub_demo_test_information.drop(index=empty_inds)

    demo_df=pd.concat([sub_demo_information.reset_index(),sub_demo_test_information.reset_index()],axis=1)

    #extract real experiment related information: 

    #if participant did not click on the arrow during the encoding a 'test_encoding_response.rt' column wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('test_encoding_response.rt' in cur_sub.columns):
        cur_sub['test_encoding_response.rt']=np.nan


    encoding_related_columns=['test_encoding_response.keys','test_encoding_response.corr','trials.thisTrialN','target_image','pair','layer','correct','test_encoding_response.rt','key_resp_end.keys']
    sub_encoding_information=cur_sub[encoding_related_columns]
    #encoding section ends with a key press of the space key: so seperate this phase by finding this space key row
    end_of_section_ind=np.where(sub_encoding_information['key_resp_end.keys']=='space')[0][0]
    sub_encoding_information=sub_encoding_information.iloc[0:end_of_section_ind]
    #remove all the rows that precede the real encoding phase: 
    empty_inds=sub_encoding_information.loc[sub_encoding_information.isnull().apply(lambda x: all(x), axis=1)].index
    sub_encoding_information=sub_encoding_information.drop(index=empty_inds).reset_index()

    #extract real experiment TEST related information: 
    #test_related_columns=['layer','correct','test_test_response.keys','test_test_response.corr','test_test_response.rt','trials_2.thisRepN','trials_2.thisTrialN','trials_2.thisN','trials_2.thisIndex','trials_2.ran','image1','image2']
    #sub_test_information=cur_sub[test_related_columns].iloc[end_of_section_ind+2:-1]


    subject_dictionary=dict()
    subject_dictionary['demographics']=sub_demographics
    subject_dictionary['demo_df']=demo_df
    subject_dictionary['encoding_df']=sub_encoding_information
    #subject_dictionary['test_df']=sub_test_information

    return subject_dictionary



In [9]:

#get all results with Encoding information - extract the arrow attention check accuracy and RT (RT is currently not usd as a criterion)

#this section extract the list of participants from the downloaded results files (and not via the workers or session list csvs) 
# - it will create the qualification_df (a table with information on the worker ids and encoding behavior of all participants that we have files for)
all_filenames=[file for file in PATH_TO_DATA.iterdir() if 'csv' in file.name and 'ENCODING' in file.name]

print(f'current csv files:\n{all_filenames}')

qualification_for_test_df=pd.DataFrame(columns=['workerID','arrow_acc','mean_arrow_RT'])
for subject_csv in all_filenames:



    #thojs is a patch for the case of saving non complete files (some are ecompletly eomptyu, and otherd dont have some important demo columns)
    if (PATH_TO_DATA / path.Path(subject_csv)).stat().st_size <= 5:
        print(f' subject: {subject_csv} did not have a demo column and thus disqualified')
        continue

    cur_sub=pd.read_csv(PATH_TO_DATA / path.Path(subject_csv))
    #get rid of participants with encoding files that are basiscly empty:
    if (not('demo_test_response.keys' in cur_sub.columns)):
        print(f' subject: {subject_csv} did not have a demo column and thus disqualified')
        continue
        


    subject_dict=process_worker_results(PATH_TO_DATA,path.Path(subject_csv))
    cur_sub_encoding=subject_dict['encoding_df']

    sname=subject_csv.name.split('_')[1] #change according to actuall format. 

    #get only attention check related info: 
    cur_sub_encoding=cur_sub_encoding[['test_encoding_response.keys','correct','test_encoding_response.rt']].copy().dropna(how = 'all')
    if len(cur_sub_encoding)==0: #empty - the participant didnt respond on the arrow at all
        arrow_acc=0
        RT=nan
    else: 
        RT=cur_sub_encoding['test_encoding_response.rt'].mean()
        arrow_acc=(cur_sub_encoding['correct']==cur_sub_encoding['test_encoding_response.keys']).mean()
    sub_df=pd.DataFrame({'workerID':sname,'arrow_acc':arrow_acc,'mean_arrow_RT':RT},index=[sname])
    qualification_for_test_df=pd.concat([qualification_for_test_df,sub_df],axis=0)

#the following part update the qualification_df with information on wether the participant id exists in the amazon workers list: 

#change participants qualifications if they exists in the workers list based on thier encoding arrow accuracy
qualification_for_test_df['in_encoding_workers_list']=nan

for curr_worker_ID in qualification_for_test_df.index:
    if curr_worker_ID in encoding_df['WorkerId'].values:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(1)
    else:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(0)


current csv files:
[WindowsPath('c:/Users/User/Desktop/visualpilot/Visual-memory-task/data/contrast/pilot_one_week/batch 1/data/ENCODING_A10A7EZUXBD0II_2022-12-13_12h05.44.260.csv'), WindowsPath('c:/Users/User/Desktop/visualpilot/Visual-memory-task/data/contrast/pilot_one_week/batch 1/data/ENCODING_A11MOA9HW11EP1_2022-12-13_15h23.07.865.csv'), WindowsPath('c:/Users/User/Desktop/visualpilot/Visual-memory-task/data/contrast/pilot_one_week/batch 1/data/ENCODING_A12TWJFR4B22X4_2022-12-13_10h43.55.280.csv'), WindowsPath('c:/Users/User/Desktop/visualpilot/Visual-memory-task/data/contrast/pilot_one_week/batch 1/data/ENCODING_A16QH3MFZ9CC4M_2022-12-13_11h04.59.200.csv'), WindowsPath('c:/Users/User/Desktop/visualpilot/Visual-memory-task/data/contrast/pilot_one_week/batch 1/data/ENCODING_A16TOOYC3QOJVE_2022-12-13_11h15.21.741.csv'), WindowsPath('c:/Users/User/Desktop/visualpilot/Visual-memory-task/data/contrast/pilot_one_week/batch 1/data/ENCODING_A1969Q0R4Y0E3J_2022-12-13_16h13.28.173.csv'), Wi

## the section below seperate between 4 types of participants: 
    * qualified and in the workers list
    * qualified and not in the workers list 
    * disqualified and in the worker list
    * disqualified and not in the worker list

In [10]:
qualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('qualified participants')
qualified_df

qualified participants


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A10A7EZUXBD0II,A10A7EZUXBD0II,1.0,1.446820,1.0
A11MOA9HW11EP1,A11MOA9HW11EP1,1.0,1.275600,1.0
A12TWJFR4B22X4,A12TWJFR4B22X4,0.8,1.823325,1.0
A16QH3MFZ9CC4M,A16QH3MFZ9CC4M,1.0,1.053660,1.0
A16TOOYC3QOJVE,A16TOOYC3QOJVE,1.0,0.924800,1.0
A1969Q0R4Y0E3J,A1969Q0R4Y0E3J,1.0,0.656220,1.0
A1DXDFRHO1A157,A1DXDFRHO1A157,1.0,0.888940,1.0
A1FCTM2O25QKL2,A1FCTM2O25QKL2,0.8,1.423750,1.0
A1G4T4PGDI1ILT,A1G4T4PGDI1ILT,1.0,1.340540,1.0
A1JWKT0IS06YKL,A1JWKT0IS06YKL,1.0,0.658260,1.0


In [11]:
qualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('qualified participants that are not in the workers list (and thus we cant add a qualification for them)')
qualified_df_not_in_workers_list

qualified participants that are not in the workers list (and thus we cant add a qualification for them)


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A3FGCKDXG0D6PB,A3FGCKDXG0D6PB,1.0,1.13338,0.0


In [12]:
disqualified_df_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('disqualified participants that are in the workers list:')
disqualified_df_in_workers_list

disqualified participants that are in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A3F9SDTQR4PEDY,A3F9SDTQR4PEDY,0.4,1.8021,1.0


In [13]:
disqualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('disqualified participants that are NOT in the workers list:')
disqualified_df_not_in_workers_list

disqualified participants that are NOT in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list


In [14]:
qualified_df['type']='qualified in workers list'
qualified_df_not_in_workers_list['type']='qualified not in workers list'
disqualified_df_in_workers_list['type']='disqualified in workers list'
disqualified_df_not_in_workers_list['type']='disqualified not in workers list'
info_from_all_downloaded_datafiles=pd.concat([qualified_df,qualified_df_not_in_workers_list,disqualified_df_in_workers_list,disqualified_df_not_in_workers_list],axis=0)
info_summary_df=info_from_all_downloaded_datafiles.groupby('type').aggregate({'arrow_acc':'count'})
info_summary_df['worker_ids']=nan
info_summary_df['worker_ids']=info_summary_df['worker_ids'].astype(object)
if 'qualified in workers list' in info_summary_df.index: 
    info_summary_df.at['qualified in workers list','worker_ids']=qualified_df.index.to_list()
if 'qualified not in workers list' in info_summary_df.index:    
    info_summary_df.at['qualified not in workers list','worker_ids']=qualified_df_not_in_workers_list.index.to_list()
if 'disqualified in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified in workers list','worker_ids']=disqualified_df_in_workers_list.index.to_list()
if 'disqualified not in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified not in workers list','worker_ids']=disqualified_df_not_in_workers_list.index.to_list()

info_summary_df

,arrow_acc,worker_ids
type,,
disqualified in workers list,1,[A3F9SDTQR4PEDY]
qualified in workers list,52,"[A10A7EZUXBD0II, A11MOA9HW11EP1, A12TWJFR4B22X..."
qualified not in workers list,1,[A3FGCKDXG0D6PB]


In [15]:
#update the workers list with the relevant qualification: 
new_shelf_dict=shelf_dict.copy()

#add the qualification for all eligible participants for the test session: 
ind_in_workers_df=[]
print('\nupdating qualifications for participants elligable for test session\n',list(qualified_df.index))
for worker_id in qualified_df.index:
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin].index[0])

print('\nupdating qualifications for participants elligable for test session that are not in the workers list\n',list(qualified_df_not_in_workers_list.index))
for worker_id in qualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#remove from experiment all those that disqualified (so they wont be able to take the test session): 
print('\nupdating qualifications for participants that were disqualified for test session and are in the workers list\n',list(disqualified_df_in_workers_list.index))
for worker_id in disqualified_df_in_workers_list.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment].index[0])
        
        if worker_id in shelf_dict.keys(): #change thier shelf entree (so they wont waste a valid csv number)
            prev_entriees=shelf_dict[worker_id]
            cur_entriees=prev_entriees.copy()
            cur_entriees[0]=999
            new_shelf_dict[worker_id]=cur_entriees
            print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#handle those that disqualified but are not in the amazon worker list: (change thier shelf entries to 999)
print('\nhandling participants that exists in the shelf but not in the amazon workers list (reseting thier shelf entriees)\n',list(disqualified_df_not_in_workers_list.index))
for worker_id in disqualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees.copy()
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#print the two relevant columns of the updated workers_list: 
print('these are all the participants that will be updated in the workers list:')
workers_df.loc[ind_in_workers_df,['Worker ID',qualification_name_for_testin,qualification_name_for_entire_experiment]]



updating qualifications for participants elligable for test session
 ['A10A7EZUXBD0II', 'A11MOA9HW11EP1', 'A12TWJFR4B22X4', 'A16QH3MFZ9CC4M', 'A16TOOYC3QOJVE', 'A1969Q0R4Y0E3J', 'A1DXDFRHO1A157', 'A1FCTM2O25QKL2', 'A1G4T4PGDI1ILT', 'A1JWKT0IS06YKL', 'A1LLBT92I9VVCQ', 'A1RSQOCZVV1FF2', 'A1Y07DW8W2UXSD', 'A24B5UF6RMHYGX', 'A25N2OW7USWS2H', 'A290Z6QAL17PQE', 'A2GUSVGAKNEGQF', 'A2L18Q7871EONK', 'A2MJQG58XN41OW', 'A2PBFKNWWIIAO8', 'A2ZHWGGGH028UZ', 'A304UJAE051J89', 'A30I2Y2HAAR3WM', 'A30O3RGJZZSMOE', 'A30ZYYC8NYLJ59', 'A324M5ZRHX7RUV', 'A39WZD2C8BPP01', 'A39XW0561NNM5C', 'A3C8E7YVLNEQKA', 'A3D9PJ37U7LCHE', 'A3HIMV8G48M87T', 'A3HN77CD9V6BLM', 'A3HP4BPBXGDM2F', 'A3R0S4QK7VY0QA', 'A3R8SCC5Q0OE7W', 'A5BMKZRGHNSRT', 'A7VA2Y4H6U31O', 'A8C3WNWRBWUXO', 'A8T1U0FM0PPWL', 'AAPVJFYLS6D4D', 'AB1UHU4WQ9HFT', 'AB6L60HXUNORT', 'AC7CDW53I98JT', 'AFFXVHHBUWW4D', 'AHXQCR64E5GUE', 'AJF954955YWZG', 'AKIT35WR8HEQ3', 'AOOU57BC3GK9J', 'AU34T9OMHN4Z4', 'AU6X92Y30ALJY', 'AV0KMH4HKQBGK', 'AYET8YADELIIA']

updating 

,Worker ID,UPDATE-eligible for memory testing 1 week,UPDATE-completed memory rep
5,A10A7EZUXBD0II,1.0,NaN
29,A11MOA9HW11EP1,1.0,NaN
49,A12TWJFR4B22X4,1.0,NaN
117,A16QH3MFZ9CC4M,1.0,NaN
119,A16TOOYC3QOJVE,1.0,NaN
154,A1969Q0R4Y0E3J,1.0,NaN
229,A1DXDFRHO1A157,1.0,NaN
254,A1FCTM2O25QKL2,1.0,NaN
273,A1G4T4PGDI1ILT,1.0,NaN
327,A1JWKT0IS06YKL,1.0,NaN


In [16]:
#save the worker list to upload (to update the qualifications) -> this is what you upload to amazon to update the qualifications of relevant participants
workers_df.to_csv(batch_data_location/'Batch_workers_for_upload_after_encoding.csv',index=False)

In [17]:
workers_df.columns

Index(['Worker ID', 'Link to Individual Worker Page',
       'Number of HITs approved or rejected - Lifetime',
       'Number of HITs approved - Lifetime', 'Your Lifetime approval rate',
       'Number of HITs approved or rejected - Last 30 days',
       'Number of HITs approved - Last 30 days',
       'Your Last 30 days approval rate',
       'Number of HITs approved or rejected - Last 7 days',
       'Number of HITs approved - Last 7 days',
       'Your Last 7 days approval rate', 'CURRENT-took part in a tapseq task',
       'UPDATE-took part in a tapseq task', 'CURRENT-participated in the past',
       'UPDATE-participated in the past',
       'CURRENT-eligible for next step memory rep',
       'UPDATE-eligible for next step memory rep',
       'CURRENT-eligible for memory testing 1 week',
       'UPDATE-eligible for memory testing 1 week',
       'CURRENT-completed memory rep', 'UPDATE-completed memory rep',
       'CURRENT-completed memory encoding 24h',
       'UPDATE-completed m

In [18]:
#this section remove participants that were assigned with an unavailble csv from the shelf (also not changing qualifications for them at all so they could come back to another encoding session in the future) 
max_csv_number=62
participants_exceeding_available_csvs=[]
for key in shelf_dict.keys():
    cur_values=shelf_dict[key]
    if (cur_values[0]>max_csv_number) and (not(cur_values[0]>=998)):
        participants_exceeding_available_csvs.append(key)
        if key in new_shelf_dict.keys():
            del new_shelf_dict[key]

print(f'the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of {max_csv_number}:\nthis probably means that they did not even see the encoding of the experiment and should be considered as never participated in our\n{participants_exceeding_available_csvs}')

the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of 62:
this probably means that they did not even see the encoding of the experiment and should be considered as never participated in our
[]


In [19]:
#this section gives 999 to participants that are in the shelf but for some reason are not in the batch_results_file (so they probably saw the experiment, but we dont know how they performed during encoding so we discard them)
for worker_id in encoding_df['WorkerId']:
    if worker_id not in qualification_for_test_df.index:
        change_msg='- is not in the shelf'
        if worker_id in new_shelf_dict.keys():
            cur_values=new_shelf_dict[worker_id]
            new_values=cur_values.copy()
            new_values[0]=999
            new_shelf_dict[worker_id]=new_values
            change_msg=f'- shelf entry changed from: {cur_values} to {new_values}'

        print(f'{worker_id} did not have an encoding file ' + change_msg)

A38KCCSGBFR26L did not have an encoding file - shelf entry changed from: [33, 1670957889055] to [999, 1670957889055]
A1NTIY7ZMBJAV2 did not have an encoding file - is not in the shelf


In [20]:
#this section find participants that have a valid shelf entree but are not in the worker list (at all) 
#going over the worker list and not the results list because it is possible these participant were entered into the shelf due to a previous batchwoekre
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id not in workers_df['Worker ID'].values:
            new_values=cur_values.copy()
            new_values[0]=999
            change_msg=f' - shelf entry changed from: {cur_values} to {new_values}'
            new_shelf_dict[worker_id]=new_values
            print(f'{worker_id} absent from WORKERS LIST ' + change_msg)


A38C544PT5L6C7 absent from WORKERS LIST  - shelf entry changed from: [2, 1670945508029, 1670945818039] to [999, 1670945508029, 1670945818039]
A2D85WX82D81GJ absent from WORKERS LIST  - shelf entry changed from: [7, 1670945930962, 1670946488104] to [999, 1670945930962, 1670946488104]
A3BI98I9V2FL0T absent from WORKERS LIST  - shelf entry changed from: [10, 1670946643322] to [999, 1670946643322]
A1DT7XOW0ISK0K absent from WORKERS LIST  - shelf entry changed from: [12, 1670946674225, 1670947134247, 1670947196277] to [999, 1670946674225, 1670947134247, 1670947196277]
A20GD3EBBTBYY2 absent from WORKERS LIST  - shelf entry changed from: [20, 1670949616488] to [999, 1670949616488]
A2DBHKZHJL03AJ absent from WORKERS LIST  - shelf entry changed from: [59, 1671006136120, 1671006207897] to [999, 1671006136120, 1671006207897]


In [21]:
#this section extracts the names of the remaining valid participants in the shelf: 
valid_participants_in_shelf_for_test=[]
valid_participants_csvnumber_in_shelf_for_test=[]
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id in workers_df['Worker ID'].values:
            valid_participants_in_shelf_for_test.append(worker_id)
            valid_participants_csvnumber_in_shelf_for_test.append(cur_values[0])

valid_shelf_participants_info_df=pd.DataFrame({'Worker ID':valid_participants_in_shelf_for_test,'numeric csv':valid_participants_csvnumber_in_shelf_for_test},index=valid_participants_in_shelf_for_test)
valid_shelf_participants_info_df

,Worker ID,numeric csv
A7VA2Y4H6U31O,A7VA2Y4H6U31O,0
A2L18Q7871EONK,A2L18Q7871EONK,1
A3R0S4QK7VY0QA,A3R0S4QK7VY0QA,3
A8C3WNWRBWUXO,A8C3WNWRBWUXO,4
A304UJAE051J89,A304UJAE051J89,5
A2PBFKNWWIIAO8,A2PBFKNWWIIAO8,6
A3R8SCC5Q0OE7W,A3R8SCC5Q0OE7W,8
A12TWJFR4B22X4,A12TWJFR4B22X4,9
AJF954955YWZG,AJF954955YWZG,15
AV0KMH4HKQBGK,AV0KMH4HKQBGK,14


In [22]:
valid_shelf_participants_info_df.join(qualification_for_test_df)

,Worker ID,numeric csv,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A7VA2Y4H6U31O,A7VA2Y4H6U31O,0,A7VA2Y4H6U31O,1.0,0.488100,1.0
A2L18Q7871EONK,A2L18Q7871EONK,1,A2L18Q7871EONK,1.0,0.629640,1.0
A3R0S4QK7VY0QA,A3R0S4QK7VY0QA,3,A3R0S4QK7VY0QA,1.0,1.940940,1.0
A8C3WNWRBWUXO,A8C3WNWRBWUXO,4,A8C3WNWRBWUXO,1.0,0.448660,1.0
A304UJAE051J89,A304UJAE051J89,5,A304UJAE051J89,1.0,0.659820,1.0
A2PBFKNWWIIAO8,A2PBFKNWWIIAO8,6,A2PBFKNWWIIAO8,1.0,1.048540,1.0
A3R8SCC5Q0OE7W,A3R8SCC5Q0OE7W,8,A3R8SCC5Q0OE7W,1.0,1.970080,1.0
A12TWJFR4B22X4,A12TWJFR4B22X4,9,A12TWJFR4B22X4,0.8,1.823325,1.0
AJF954955YWZG,AJF954955YWZG,15,AJF954955YWZG,0.8,0.630375,1.0
AV0KMH4HKQBGK,AV0KMH4HKQBGK,14,AV0KMH4HKQBGK,1.0,1.146440,1.0


In [23]:
#this is the updated shelf: copy from here and place in the shelf location in pavlovia.org
import json
json.dumps(new_shelf_dict)

'{"A7VA2Y4H6U31O": [0, 1670945335008], "A2L18Q7871EONK": [1, 1670945472029], "A38C544PT5L6C7": [999, 1670945508029, 1670945818039], "A2C4Y3H6AI7TOC": [999, 999, 999], "A3R0S4QK7VY0QA": [3, 1670945605406], "A8C3WNWRBWUXO": [4, 1670945758899], "A37INK0K3GRFGL": [999, 999, 999], "A27DJWJG2RN67O": [999, 999, 999], "A304UJAE051J89": [5, 1670945807462], "A2PBFKNWWIIAO8": [6, 1670945922618], "A2D85WX82D81GJ": [999, 1670945930962, 1670946488104], "A3R8SCC5Q0OE7W": [8, 1670946068317], "A2K454H8X0MQ6": [999, 999, 999], "A12TWJFR4B22X4": [9, 1670946235280], "A29UY77DJOCTRB": [999, 999, 999], "A3FGCKDXG0D6PB": [999, 1670946371142, 1670946819241, 1670947949899, 1670949700945], "A3BI98I9V2FL0T": [999, 1670946643322], "A1DT7XOW0ISK0K": [999, 1670946674225, 1670947134247, 1670947196277], "A2UOURWPFGV8A4": [999, 999, 999], "AMW28M2VV2677": [999, 999, 999], "AJF954955YWZG": [15, 1670946923359], "AV0KMH4HKQBGK": [14, 1670947073106], "A3ECPU0FSUCJCM": [999, 999, 999], "A1LLBT92I9VVCQ": [16, 1670947510184,

In [46]:
clean_shelf_after_test=False #change to true if you run this code after completing both encoding and test sessions and want to also resert participants that didnt come back at all... 
if clean_shelf_after_test: 


    allowed_interval_in_hours=24
    allowed_jitter=3
    allowed_interval_in_ms=[allowed_interval_in_hours-allowed_jitter,allowed_interval_in_hours+allowed_jitter]*3600*1000


    new_shelf_dict=shelf_dict.copy()
    changed_keys_list=[]
    for key in new_shelf_dict.keys():
        cur_entries=new_shelf_dict[key]
        if len(cur_entries)==2: 
            cur_entries[0]=999
            changed_keys_list.append(key)

        if len(cur_entries)>2:
            encoding_time=cur_entries[1]
            last_entree=cur_entries[-1]
            if (last_entree - encoding_time) < allowed_interval_in_ms[0]:
                cur_entries[0]=999
                changed_keys_list.append(key)


        new_shelf_dict[key]=cur_entries

    print('changed entrees for:',changed_keys_list)

    ##### print the updated shelf dictionary so you can copy it from the cell output and paste in the shelf:  (change the shelf only if you run this code after both encoding and TEST has ended) ####
    new_shelf_dict
